# Amazon Lookout for Vision Python SDK

이 노트북에서는 Amazon Lookout for Vision Python SDK에 대해 설명합니다. 해당 서비스와 상호 작용하는 프로그래밍 방식을 제공하고 아래와 같이 서비스를 보완하는 다양한 Helper 함수를 추가합니다.

* Manifest 파일 생성하기
* Manifest 파일을 S3에 푸시하기
* 이미지 크기가 서비스 규격과 맞는지 확인하기
* 이미지 크기를 조정해야하는지 이미지 크기 확인하기
* 최적 크기로 알맞게 이미지 크기 조정하기
* 적절한 구조로 S3에 이미지 업로드하기

**필요 조건**

로컬 인스턴스에 이미지가 존재해야 합니다. 비정상 이미지는 *bad*라는 폴더에, 정상 이미지는 *good*라는 폴더에 저장해야합니다. 또한 허용되는 포맷은 jpeg, jpg와 png입니다. 다음 URL은 훈련과 검증을 위한 이미지 할당량 / 제한을 설명합니다. --> https://docs.aws.amazon.com/lookout-for-vision/latest/developer-guide/limits.html 

## 모델 훈련하기

먼저 필요한 몇 가지 일반 변수를 설정해 보겠습니다.

* input_bucket: 모델 훈련을 위해 이미지를 보관할 S3 버킷
* project_name: Amazon Lookout for Vision 프로젝트의 고유한 이름
* model_version: 배포할 모델 버전 (참고: 처음 시작할 경우 "1"이 기본값) 
* output_bucket: 모델과 추론 결과가 저장되는 버킷 (input_bucket과 동일할 수 있음) 
* input_prefix: S3에서 추론을 실행하는 경우 이것은 예측하려는 이미지의 키 값입니다
* output_prefix: 이것은 예측값이 저장될 S3 키 값입니다

In [1]:
# 훈련 & 추론
input_bucket = "lfv-s3-bucket-<youralias>-MMDDYY-v1"
project_name = "circuitproject"
model_version = "1" # 최초로 시작할 경우 이것을 1로 두세요.
# 추론
output_bucket = input_bucket # INPUT_BUCKET과 동일할 수 있습니다.
input_prefix = "extra_images/" # batch_predict에서 사용됩니다.
output_prefix = input_prefix # batch_predict에서 사용됩니다.

In [2]:
# pip를 사용하여 SDK를 설치합니다.
# !pip install lookoutvision

In [3]:
# 시작하는데 필요한 모든 라이브러리를 임포트합니다.
import os
import boto3
import pandas as pd

from lookoutvision.image import Image
from lookoutvision.manifest import Manifest
from lookoutvision.lookoutvision import LookoutForVision
from lookoutvision.metrics import Metrics

In [4]:
def upload_directory(s3_client, path, bucket, s3_path=""):
    for root, dirs, files in os.walk(path):
        for file in files:
            s3_client.upload_file(os.path.join(root, file), bucket, s3_path + file)

필요한 클래스를 인스턴스화하세요.

* 로컬 이미지와 상호 작용할 Image
* Manifest 파일을 생성하고 푸시할 Manifest
* 모델 메트릭을 살펴보고 비교할 Metrics
* 서비스와 상호 작용하는 기본 클래스 LookoutForVision 

In [5]:
s3_client = boto3.client("s3")
_ = s3_client.create_bucket(Bucket=input_bucket)

In [6]:
img = Image()

In [7]:
mft = Manifest(
    bucket=input_bucket
    , s3_path=project_name
    , datasets=["training", "validation"])

In [8]:
l4v = LookoutForVision(project_name=project_name)

Project circuitproject does not exist yet...use the create_project() method to set up your first project


In [9]:
met = Metrics(project_name=project_name)

In [10]:
# 프로젝트가 존재하지 않는 경우 생성하세요.
p = l4v.create_project()
# print(p)

Creating the project: circuitproject


In [11]:
!mkdir -p good && mkdir -p bad

!cp -R circuitboard/train/normal/ good/
!cp -R circuitboard/test/normal/ good/

!cp -R circuitboard/train/anomaly/ bad/
!cp -R circuitboard/test/anomaly/ bad/

In [12]:
# 로컬 이미지가 서비스 규격과 맞는지 확인하세요 
sizes = img.check_image_sizes(verbose=False)
print(sizes)

{'good': {'no_of_images': 40, 'compliant_images': 40, 'compliant': True}, 'bad': {'no_of_images': 40, 'compliant_images': 40, 'compliant': True}}


In [13]:
# 모든 이미지 크기가 동일한지 확인하세요
shapes = img.check_image_shapes(verbose=True)
print(shapes)

{'good': {'no_of_images': 40, 'compliant': 40, 'status': 'Image sizes are equal!', 'min_image_shape': (2667, 4000, 3), 'image_metadata': {'good/test-normal_8.jpg': (2667, 4000, 3), 'good/test-normal_9.jpg': (2667, 4000, 3), 'good/train-normal_14.jpg': (2667, 4000, 3), 'good/test-normal_11.jpg': (2667, 4000, 3), 'good/train-normal_5.jpg': (2667, 4000, 3), 'good/train-normal_4.jpg': (2667, 4000, 3), 'good/test-normal_10.jpg': (2667, 4000, 3), 'good/train-normal_15.jpg': (2667, 4000, 3), 'good/train-normal_17.jpg': (2667, 4000, 3), 'good/test-normal_12.jpg': (2667, 4000, 3), 'good/train-normal_6.jpg': (2667, 4000, 3), 'good/train-normal_7.jpg': (2667, 4000, 3), 'good/test-normal_13.jpg': (2667, 4000, 3), 'good/train-normal_16.jpg': (2667, 4000, 3), 'good/train-normal_12.jpg': (2667, 4000, 3), 'good/test-normal_17.jpg': (2667, 4000, 3), 'good/train-normal_3.jpg': (2667, 4000, 3), 'good/train-normal_2.jpg': (2667, 4000, 3), 'good/test-normal_16.jpg': (2667, 4000, 3), 'good/train-normal_13.j

In [14]:
# 만약 서비스 규격과 맞지 않거나 모든 이미지 크기가 동일하지 않다면 조정하세요.
# 참고: 접두사를 꼭 지정할 필요 없습니다. 지정할 경우 신규 폴더가 해당 명칭에 맞게 생성됩니다.
# rescaled_good과 rescaled_bad, 접두사가 없으면 원본 이미지를 덮어 씁니다.
resc = img.rescale(prefix="rescaled_")
print(resc)

No rescaling needed!
{'rescaled_good': 'Ok', 'rescaled_bad': 'Ok'}


In [15]:
# 크기를 조정한 폴더로 재확인하세요. (조정한 경우)
# sizes = img.check_image_sizes(prefix="rescaled_", verbose=False)
# print(sizes)

In [16]:
# 크기가 조정한 폴더로 다시 확인하세요. (조정한 경우) 
# shapes = img.check_image_shapes(prefix="rescaled_", verbose=True)
# print(shapes)

이미지를 준비하고 모두 동일한 크기로 만들고 서비스 규격에 맞춘 다음 S3 버킷에 업로드할 수 있습니다. Image() 클래스가 적절히 업로드하므로 더 이상 구조에 신경쓸 필요 없습니다. 

In [17]:
_ = img.upload_from_local(
    bucket=input_bucket
    , s3_path=project_name
    , train_and_test=True
    , test_split=0.5
    , prefix="") 

이제 이미지를 S3에 저장했므로 Manifest() 클래스를 사용하여 Manifest 파일을 생성하고 이미지 폴더를 저장한, 동일한 S3 위치에 푸시할 수 있습니다.

In [18]:
mft_resp = mft.push_manifests()
# print(mft_resp)

S3의 Manifest 파일을 기반으로 Lookout for Vision 데이터셋을 생성합니다

In [19]:
dsets = l4v.create_datasets(mft_resp, wait=True)
print(dsets)

Creating dataset(s): --!
{'training': {'DatasetType': 'train', 'CreationTimestamp': datetime.datetime(2021, 4, 9, 16, 42, 37, 124000, tzinfo=tzlocal()), 'Status': 'CREATE_COMPLETE', 'StatusMessage': 'Dataset created.'}, 'validation': {'DatasetType': 'test', 'CreationTimestamp': datetime.datetime(2021, 4, 9, 16, 42, 37, 383000, tzinfo=tzlocal()), 'Status': 'CREATE_COMPLETE', 'StatusMessage': 'Dataset created.'}}


모델을 훈련할 준비가 되었습니다.

In [20]:
_ = l4v.fit(
    output_bucket=output_bucket
    , model_prefix="mymodel_"
    , wait=True)

Model training started: ----------------------------------------!


그리고 최종 배포합시다.

In [21]:
_ = l4v.deploy(
    model_version=model_version
    , wait=True)

Model will be hosted now
---------!
Your model is now hosted!


## 모델 메트릭 표시하기 

모델 메트릭을 확인할 때 두 가지 유형의 *Metrics* 클래스를 사용할 수 있습니다.

* 모델 한 개에 대해 메트릭을 표시하거나
* 동일한 프로젝트의 모든 모델에 대한 메트릭을 표시합니다.

In [22]:
# 모델 한 개 
columns = ["CreationTimestamp", "Status", "StatusMessage", "Performance"]
met.describe_model(model_version=model_version)[columns]

,CreationTimestamp,Status,StatusMessage,Performance
F1Score,2021-04-09 16:44:23.114000+09:00,HOSTED,The model is running.,0.9
Precision,2021-04-09 16:44:23.114000+09:00,HOSTED,The model is running.,0.9
Recall,2021-04-09 16:44:23.114000+09:00,HOSTED,The model is running.,0.9


In [23]:
# 동일한 프로젝트의 모든 모델
met.describe_models()[columns]

,CreationTimestamp,Status,StatusMessage,Performance
F1Score,2021-04-09 16:44:23.114000+09:00,HOSTED,The model is running.,0.9
Precision,2021-04-09 16:44:23.114000+09:00,HOSTED,The model is running.,0.9
Recall,2021-04-09 16:44:23.114000+09:00,HOSTED,The model is running.,0.9


## 추론

### Batch Transform 기능을 사용하면 Amazon S3 / 로컬에 저장된 데이터셋에 대해 예측을 실행할 수 있습니다. 
배치 변환 작업은 배치 데이터셋에서 추론을 실행하고 그에 따른 추론 결과를 S3 / 로컬에 저장합니다. 

데이터 / 이미지가 s3에 있는 배치 예측의 경우 함수에 대한 입력값으로 아래 정보를 제공하세요.
  1. model_version = 모델 버전을 입력하거나 기본값인 모델 버전 1을 사용합니다. 
  2. input_bucket = 입력 이미지 (정상 / 이상치 예측이 필요한)가 있는 입력 버킷 이름입니다.
  3. input_prefix = 입력 이미지가 있는 s3 경로의 폴더 이름 / 키 값 (필요하다면). 이 경우 예제에서 언급한대로 끝에 슬래시 ("/")를 넣어야합니다. 
  4. output_bucket = 예측 결과를 json 파일에 저장할 출력 버킷 이름입니다. 출력 json 파일의 이름은 image_name.json이 됩니다. 
  5. output_prefix = 예측 출력 파일을 저장할 s3 경로의 폴더 이름 / 키 값 (필요하다면). 이 경우 예제에서 언급한대로 끝에 슬래시 ("/")를 입력했는지 확인하세요.
  6. content_type = "image/jpeg"


In [24]:
upload_directory(s3_client, "circuitboard/extra_images", input_bucket, s3_path=input_prefix)

In [25]:
_ = l4v.batch_predict(
    model_version=model_version
    , input_bucket=input_bucket
    , input_prefix=input_prefix 
    , output_bucket=output_bucket
    , output_prefix=output_prefix 
    , content_type="image/jpeg")

데이터 / 이미지가 로컬에 있는 배치 예측의 경우 함수에 대한 입력값으로 아래 정보를 제공하세요.

1. model_version = 모델 버전을 입력하거나 기본값으로 모델 버전 1을 사용합니다. 
2. local_path = 입력 이미지 (정상 / 이상치 예측이 필요한)가 있는 로컬 경로.
3. content_type = "image / jpeg"

In [26]:
predicted = l4v._batch_predict_local(
    local_path="circuitboard/extra_images"
    , model_version=model_version
    , content_type="image/jpeg")

In [27]:
predicted = pd.json_normalize(predicted["predicted_result"])
predicted.head()

,IsAnomalous,Confidence,Source.Type
0,True,0.973909,direct
1,False,0.551062,direct
2,False,0.914603,direct
3,True,0.998862,direct
4,False,0.311159,direct


### 실시간으로 예측하려면 아래 입력값으로 predict 메서드를 호출하세요.
 1. model_version = 모델 버전을 입력하거나 기본값으로 모델 버전 1을 사용합니다.  
 2. local_file = 입력 이미지 (정상 / 이상치 예측이 필요한)가 있는 로컬 경로.
 3. bucket = 입력 이미지 (정상 / 이상치 예측이 필요한)가 있는 입력 버킷 이름입니다.
 4. key = 이미지의 키 값 (아래 예에 언급된 것처럼 정확한 파일 이름을 포함해야 함)
 5. content_type = "image/jpeg"

In [28]:
# 이미지가 로컬 경로에 있을 때. 로컬 디렉토리와 파일 명으로 로컬 파일 경로를 변경하십시오.
l4v.predict(local_file="circuitboard/extra_images/extra_images-anomaly_1.jpg")

{'Source': {'Type': 'direct'},
 'IsAnomalous': True,
 'Confidence': 0.9930121898651123}

In [29]:
# 이미지가 로컬 경로에 있을 때. 로컬 디렉토리와 파일 명으로 로컬 파일 경로를 변경하십시오.
l4v.predict(local_file="circuitboard/extra_images/extra_images-normal_1.jpg")

{'Source': {'Type': 'direct'},
 'IsAnomalous': False,
 'Confidence': 0.31115949153900146}

In [30]:
# 이미지가 s3에 있을 때. 키 값과 파일 이름으로 s3 버킷을 변경하세요.
l4v.predict(
    bucket=input_bucket
    , key=input_prefix + "extra_images-anomaly_2.jpg")

{'Source': {'Type': 'direct'},
 'IsAnomalous': False,
 'Confidence': 0.41652774810791016}

### 동일한 프로젝트의 모델을 재교육하려면 다음 단계를 따라야합니다. 

1. 새로운 이미지로 신규 / 업데이트된 Manifest 파일 생성하기

2. 기존 데이터셋 업데이트하기 (훈련과 테스트 모두)

3. 업데이트한 데이터셋으로 새로운 버전의 모델 훈련하기 

In [31]:
!cp -R circuitboard/extra_images/extra_images-anomaly_*.jpg bad/
!cp -R circuitboard/extra_images/extra_images-normal_*.jpg good/

In [32]:
input_bucket_new = input_bucket[:-1] + "2"
_ = s3_client.create_bucket(Bucket=input_bucket_new)

In [33]:
## 훈련시킬 최신 입력 이미지에 대한 버킷을 정의한 다음 다음과 같이 Manifest 메서드를 초기화합니다.
## 동일한 버킷에 신규 / 업데이트된 이미지가 있는 경우 본 단계는 생략할 수 있습니다. 
mft_retrain = Manifest(
    bucket=input_bucket_new
    , s3_path=project_name
    , datasets=["training", "validation"])

In [34]:
# 로컬에 재훈련을 위한 신규 / 업데이트된 이미지가 있는 경우 다음과 같이 동일한 이미지를 s3로 가져올 수 있습니다. 
_ = img.upload_from_local(
    bucket=input_bucket_new
    , s3_path=project_name
    , train_and_test=True
    , test_split=0.5
    , prefix="") 

In [35]:
# 이제 신규 데이터셋에 대한 Manifest 파일을 만듭니다.
mft_resp_new = mft_retrain.push_manifests()

In [36]:
# 새롭게 만든 Manifest 파일로 데이터셋을 업데이트합니다.
l4v.update_datasets(mft_resp_new)

Creating dataset(s): ---!


{'training': {'DatasetType': 'train',
  'CreationTimestamp': datetime.datetime(2021, 4, 9, 17, 49, 55, 776000, tzinfo=tzlocal()),
  'Status': 'CREATE_COMPLETE',
  'StatusMessage': 'Dataset created.'},
 'validation': {'DatasetType': 'test',
  'CreationTimestamp': datetime.datetime(2021, 4, 9, 17, 49, 56, 353000, tzinfo=tzlocal()),
  'Status': 'CREATE_COMPLETE',
  'StatusMessage': 'Dataset created.'}}

In [37]:
# 신규 모델 훈련 생성을 시작합니다. 이번에는 업데이트된 데이터셋이 필요합니다. 
_ = l4v.fit(output_bucket=input_bucket_new)

Model training started: ----------------------------------------------!


### 작업을 완료한 다음 모델을 중지하세요.
모델 버전을 언급하지 않으면 기본적으로 모델 버전 1이 중지됩니다. 

In [38]:
# 모델 버전을 언급하지 않을 때
l4v.stop_model()

Stopping model version 1 for project circuitproject
Model will be stopped now
-----!
Your model is now stopped!
Status: STOPPING_HOSTING


{'ResponseMetadata': {'RequestId': 'e75ecb3d-1a9e-431d-a0ca-a46eefc3e55a',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'x-amzn-requestid': 'e75ecb3d-1a9e-431d-a0ca-a46eefc3e55a',
   'x-xss-protection': '1; mode=block',
   'strict-transport-security': 'max-age=31540000; includeSubDomains',
   'x-frame-options': 'DENY',
   'x-content-type-options': 'nosniff',
   'date': 'Fri, 09 Apr 2021 09:36:25 GMT',
   'content-type': 'application/json',
   'content-length': '29'},
  'RetryAttempts': 0},
 'Status': 'STOPPING_HOSTING'}

In [39]:
# 특정 모델 버전을 언급 할 때 
# new_model_version = "2"
# l4v.stop_model(model_version=new_model_version)

In [40]:
!rm -rf good && rm -rf bad